# Delay Embedding and the MFPT

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pyedgar
from pyedgar.data_manipulation import tlist_to_flat, flat_to_tlist, delay_embed, lift_function

%matplotlib inline

## Load Data and set Hyperparameters
We first load in the pre-sampled data.  The data consists of 400 short trajectories, each with 30 datapoints.  The precise sampling procedure is described in "Galerkin Approximation of Dynamical Quantities using Trajectory Data" by Thiede et al.  Note that this is a smaller dataset than in the paper.  We use a smallar dataset to ensure the diffusion map basis construction runs in a reasonably short time.

### Set Hyperparameters
Here we specify a few hyperparameters.  Thes can be varied to study the behavior of the scheme in various limits by the user.

In [2]:
ntraj = 300
trajectory_length = 20

### Load and format the data

In [3]:
trajs = np.load('data/muller_brown_trajs.npy')[:ntraj, :trajectory_length, 1] # Raw trajectory
stateA = (trajs > 1.15).astype('float')
stateB = (trajs < 0.15).astype('float')
print("Data shape: ", trajs.shape)

# Convert to list of trajectories format
trajs = [traj_i.reshape(-1, 1) for traj_i in trajs]
stateA = [A_i for A_i in stateA]
stateB = [B_i for B_i in stateB]

# We also define a new trajectory to be used for plotting
traj_2d = np.load('data/muller_brown_trajs.npy')[:ntraj, :trajectory_length, :2].reshape(-1 ,2)

Data shape:  (300, 20)


We also convert the data into the flattened format.  This converts the data into a 2D array, which allows the data to be passed into many ML packages that require a two-dimensional dataset.  In particular, this is the format accepted by the Diffusion Atlas object.  Trajectory start/stop points are then stored in the traj_edges array.

In [4]:
flattened_trajs, traj_edges = tlist_to_flat(trajs)
flattened_stateA = np.hstack(stateA)
flattened_stateB = np.hstack(stateB)
print("Flattened Shapes are: ", flattened_trajs.shape, flattened_stateA.shape, flattened_stateB.shape,)

Flattened Shapes are:  (6000, 1) (6000,) (6000,)


## Construct DGA MFPT by increasing lag times
We first construct the MFPT with increasing lag times.

In [5]:
# # Build the basis set
# diff_atlas = pyedgar.basis.DiffusionAtlas.from_sklearn(alpha=0, k=500, bandwidth_type='-1/d', epsilon='bgh_generous')
# diff_atlas.fit(flattened_trajs)
# flat_basis = diff_atlas.make_dirichlet_basis(50, in_domain=(1. - flattened_stateA))
# basis = flat_to_tlist(flat_basis, traj_edges)

In [6]:
# # Perform DGA calculation
# mfpt_BA_lags = []
# for lag in range(1,20):
#     mfpt = pyedgar.galerkin.compute_mfpt(basis, stateA, lag=lag)
#     mfpt_BA = np.mean(np.array(mfpt).ravel() * flattened_stateB) / np.mean(stateB)
# #     pi = pyedgar.galerkin.compute_change_of_measure(basis_no_boundaries, lag=lag)
#     mfpt_BA_lags.append(mfpt_BA)

## Construct DGA MFPT with increasing Delay Embedding
We now construct the MFPT using delay embedding.  To accelerate the process, we will only use every fifth value of the delay length.

In [10]:
import importlib
importlib.reload(pyedgar)

mfpt_BA_embeddings = []
for lag in range(14, 15, 5):
    # Perform delay embedding
    debbed_traj = delay_embed(trajs, n_embed=lag)
    lifted_A = lift_function(stateA, n_embed=lag)
    lifted_B = lift_function(stateB, n_embed=lag)
    
    flat_debbed_traj, embed_edges = tlist_to_flat(debbed_traj)
    flat_lifted_A = np.vstack(lifted_A)
        
    # Build the basis 
    diff_atlas = pyedgar.basis.DiffusionAtlas.from_sklearn(alpha=0, k=500, bandwidth_type='-1/d',
                                                           epsilon='bgh_generous')
    diff_atlas.fit(flat_debbed_traj)
    flat_deb_basis = diff_atlas.make_dirichlet_basis(50, in_domain=(1. - flat_lifted_A))
    deb_basis = flat_to_tlist(flat_deb_basis, embed_edges)
    
    # Construct the Estimate
    print(np.shape(deb_basis), np.shape(lifted_A), type(deb_basis), type(lifted_A))
    deb_mfpt = pyedgar.galerkin.compute_mfpt(deb_basis, lifted_A, lag=1)
    deb_mfpt_BA = np.mean(np.array(deb_mfpt).ravel() * np.array(lifted_B).ravel()) / np.mean(stateB)
    mfpt_BA_embeddings.append(deb_mfpt_BA)

(300, 6, 50) (300, 6) <class 'list'> <class 'list'>
------------
1
[   0    6   12   18   24   30   36   42   48   54   60   66   72   78
   84   90   96  102  108  114  120  126  132  138  144  150  156  162
  168  174  180  186  192  198  204  210  216  222  228  234  240  246
  252  258  264  270  276  282  288  294  300  306  312  318  324  330
  336  342  348  354  360  366  372  378  384  390  396  402  408  414
  420  426  432  438  444  450  456  462  468  474  480  486  492  498
  504  510  516  522  528  534  540  546  552  558  564  570  576  582
  588  594  600  606  612  618  624  630  636  642  648  654  660  666
  672  678  684  690  696  702  708  714  720  726  732  738  744  750
  756  762  768  774  780  786  792  798  804  810  816  822  828  834
  840  846  852  858  864  870  876  882  888  894  900  906  912  918
  924  930  936  942  948  954  960  966  972  978  984  990  996 1002
 1008 1014 1020 1026 1032 1038 1044 1050 1056 1062 1068 1074 1080 1086
 1092 1098

LinAlgError: Matrix is singular.

In [ ]:
plt.scatter(range(4, 15, 5), mfpt_BA_embeddings)

## Compare against reference

To compare against the reference values, we will interpolate the reference onto the datapoints usingy scipy's interpolate package.

In [ ]:
import scipy.interpolate as spi

spline = spi.RectBivariateSpline(xgrid, ygrid, ref_comm.T)
ref_comm_on_data = np.array([spline.ev(c[0], c[1]) for c in flattened_trajs[:,:2]])
ref_comm_on_data[ref_comm_on_data < 0.] = 0.
ref_comm_on_data[ref_comm_on_data > 1.] = 1.

A comparison of our estimate with the True committor.  While the estimate is good, we systematically underestimate the committor near (0, 0.5).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,3.5), sharex=True, sharey=True)
(ax1, ax2, ax3) = axes
SC = ax1.scatter(flattened_trajs[:,0], flattened_trajs[:,1], c=ref_comm_on_data, vmin=0., vmax=1., s=3)
plt.colorbar(SC, ax=ax1)
SC = ax2.scatter(flattened_trajs[:,0], flattened_trajs[:,1], c=np.array(g).ravel(), vmin=0., vmax=1., s=3)
plt.colorbar(SC, ax=ax2)
SC = ax3.scatter(flattened_trajs[:,0], flattened_trajs[:,1], c=np.array(g).ravel() -ref_comm_on_data, 
                 vmin=-1, vmax=1, s=3, cmap='bwr')
plt.colorbar(SC, ax=ax3)


# ax1.set_aspect('equal')
ax2.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('True Committor')
ax2.set_title('DGA Estimate')
ax3.set_title('Estimate - True')
plt.tight_layout(pad=-1.)
for ax in axes:
    ax.set_aspect('equal')